In [1]:
from dask.distributed import Client
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:10218 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.92 GB


In [3]:
!pip install nycflights13

  Using cached https://files.pythonhosted.org/packages/a1/6a/ce6fe2de399a54e1fc4c4b60c61987854974b936bab6d0f6444bc76939db/nycflights13-0.0.3.tar.gz
  Created wheel for nycflights13: filename=nycflights13-0.0.3-cp37-none-any.whl size=8732762 sha256=72aa4fcfba658c1a4f09b6a26e6e302446fe11ca0e04ab8b73a80943b28a4b94
  Stored in directory: C:\Users\jainh\AppData\Local\pip\Cache\wheels\50\fe\ad\c6adafdf6e59183110670b4b14fb582c20b82a0fed449348cb
Successfully built nycflights13


In [4]:
import dask
import dask.dataframe as dd
from nycflights13 import flights
df = dd.from_pandas(flights, npartitions=4)
df.head()


C:\Users\jainh\Anaconda3\lib\site-packages\distributed\worker.py:3239: UserWarning: Large object of size 11.35 MB detected in task graph: 
  (       year  month  day  dep_time  sched_dep_time ... 19 columns], 5)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [5]:
print("Before performing the pandas operation: {}".format(flights['dep_delay'].isnull().sum()))


Before performing the pandas operation: 8255


In [6]:
flights.dropna(subset=['dep_delay'], how ='all', inplace = True)


In [7]:
print("After cleaning the dataframe: {}".format(flights['dep_delay'].isnull().sum()))


After cleaning the dataframe: 0


In [8]:
print(flights.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328521 entries, 0 to 336769
Data columns (total 19 columns):
year              328521 non-null int64
month             328521 non-null int64
day               328521 non-null int64
dep_time          328521 non-null float64
sched_dep_time    328521 non-null int64
dep_delay         328521 non-null float64
arr_time          328063 non-null float64
sched_arr_time    328521 non-null int64
arr_delay         327346 non-null float64
carrier           328521 non-null object
flight            328521 non-null int64
tailnum           328521 non-null object
origin            328521 non-null object
dest              328521 non-null object
air_time          327346 non-null float64
distance          328521 non-null int64
hour              328521 non-null int64
minute            328521 non-null int64
time_hour         328521 non-null object
dtypes: float64(5), int64(9), object(5)
memory usage: 50.1+ MB
None


In [15]:
dep_delay_mean = df['dep_delay'].mean().compute()
dep_delay_mean

12.639070257304708

In [16]:
dep_delay_std = df['dep_delay'].std().compute()
dep_delay_std

40.21006089212995